In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from xgboost import XGBRegressor
from datetime import datetime
import json
import unicodedata

df = pd.read_csv("../dataset_mare_tratado.csv")

print("✅ Dataset carregado:")
print(df.head(10))

✅ Dataset carregado:
         praia  latitude  longitude        data   hora  altura  hora_decimal  \
0  barra_norte       0.0        0.0  2025-01-01  04:04    0.04      4.066667   
1  barra_norte       0.0        0.0  2025-01-01  10:27    3.41     10.450000   
2  barra_norte       0.0        0.0  2025-01-01  16:21    0.62     16.350000   
3  barra_norte       0.0        0.0  2025-01-01  22:34    3.52     22.566667   
4  barra_norte       0.0        0.0  2025-02-01  05:14    0.11      5.233333   
5  barra_norte       0.0        0.0  2025-02-01  11:51    3.48     11.850000   
6  barra_norte       0.0        0.0  2025-02-01  17:36    0.61     17.600000   
7  barra_norte       0.0        0.0  2025-02-01  23:59    3.38     23.983333   
8  barra_norte       0.0        0.0  2025-03-01  04:51   -0.03      4.850000   
9  barra_norte       0.0        0.0  2025-03-01  11:08    3.50     11.133333   

   mes   ano  dia_mes  semana_ano dia_da_semana  
0    1  2025        1           1     Wednesday 

In [ ]:
df['hora_rad'] = 2 * np.pi * df['hora_decimal'] / 24

# Features cíclicas
df['hora_sin'] = np.sin(df['hora_rad'])
df['hora_cos'] = np.cos(df['hora_rad'])

In [ ]:
df['data'] = pd.to_datetime(df['data'], errors='coerce')

df['dia_semana_num'] = df['data'].dt.weekday  

features = ['latitude', 'longitude', 'mes', 'dia_mes', 'dia_semana_num', 'hora_sin', 'hora_cos']
X = df[features]
y = df['altura']

In [ ]:
model = XGBRegressor(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)

model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.05, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=6,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=500,
             n_jobs=None, num_parallel_tree=None, ...)

In [25]:
y_pred = model.predict(X_test)

rmse = mean_squared_error(y_test, y_pred, squared=False)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"RMSE: {rmse:.2f} metros")
print(f"MAE: {mae:.2f} metros")
print(f"R²: {r2:.2f}")

RMSE: 1.41 metros
MAE: 1.10 metros
R²: -0.00


c:\Users\maria\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [ ]:
praias_unicas = df['praia'].unique()
nomes_praias_map = {praia: " ".join([p.capitalize() for p in praia.split("_")]) 
                    for praia in praias_unicas}

def normalizar_nome_praia(nome_usuario, mapa=nomes_praias_map):
    nome_norm = nome_usuario.strip().lower()
    
    for dataset_name, user_name in mapa.items():
        if nome_norm == user_name.lower():
            return dataset_name
    
    return None

nome_input = "Porto de Suape"
nome_dataset = normalizar_nome_praia(nome_input)

print(f"Nome digitado: {nome_input} → Nome no dataset: {nome_dataset}")

Nome digitado: Porto de Suape → Nome no dataset: porto_de_suape


In [ ]:
def normalizar_nome_praia(nome):
    """
    Normaliza nomes de praia (minúsculo, remove acentos e troca espaços por _)
    """
    nome = nome.lower()
    nome = ''.join(c for c in unicodedata.normalize('NFKD', nome) if not unicodedata.combining(c))
    nome = nome.replace(' ', '_')
    return nome

def criar_features(latitude, longitude, data_str, hora_str):
    data = pd.to_datetime(data_str)
    hora_decimal = int(hora_str.split(':')[0]) + int(hora_str.split(':')[1])/60

    hora_rad = 2 * np.pi * hora_decimal / 24
    hora_sin = np.sin(hora_rad)
    hora_cos = np.cos(hora_rad)

    mes = data.month
    mes_rad = 2 * np.pi * mes / 12
    mes_sin = np.sin(mes_rad)
    mes_cos = np.cos(mes_rad)

    dia_sem = data.weekday()  
    dia_sem_rad = 2 * np.pi * dia_sem / 7
    dia_sin = np.sin(dia_sem_rad)
    dia_cos = np.cos(dia_sem_rad)

    return pd.DataFrame([{
        'latitude': latitude,
        'longitude': longitude,
        'hora_sin': hora_sin,
        'hora_cos': hora_cos,
        'mes_sin': mes_sin,
        'mes_cos': mes_cos,
        'dia_sin': dia_sin,
        'dia_cos': dia_cos
    }])

def prever_mare_auto(nome_usuario, data_str, df, model, hora=None, json_path="../public/praias.json"):
    nome_dataset = normalizar_nome_praia(nome_usuario)

    if nome_dataset in df['praia'].unique():
        df_praia = df[df['praia'] == nome_dataset].iloc[0]  # pega primeira linha para lat/lon
        latitude = df_praia['latitude']
        longitude = df_praia['longitude']
    else:
        with open(json_path, 'r', encoding='utf-8') as f:
            praias_json = json.load(f)
        praia_info = next((p for p in praias_json if normalizar_nome_praia(p['praia']) == nome_dataset), None)
        if praia_info is None:
            raise ValueError(f"Praia '{nome_usuario}' não encontrada no dataset nem no JSON.")
        latitude = praia_info['lat']
        longitude = praia_info['lon']

    if hora is None:
        horas = ['00:00','04:00','08:00','12:00','16:00','20:00']
    elif isinstance(hora, str):
        horas = [hora]
    else:
        horas = hora

    resultados = []
    for h in horas:
        X_pred = criar_features(latitude, longitude, data_str, h)
        altura_prevista = model.predict(X_pred)[0]
        resultados.append({
            'praia': nome_usuario,
            'data': data_str,
            'hora': h,
            'altura_prevista': altura_prevista
        })
    return pd.DataFrame(resultados)

In [76]:
df_result = prever_mare_auto('Itapuama', '2025-01-01', df, model, hora='14:30')
print(df_result)

      praia        data   hora  altura_prevista
0  Itapuama  2025-01-01  14:30         1.232759


In [77]:
df_result = prever_mare_auto('Itapuama', '2025-01-01', df, model)
print(df_result)

      praia        data   hora  altura_prevista
0  Itapuama  2025-01-01  00:00         1.610833
1  Itapuama  2025-01-01  04:00         1.607059
2  Itapuama  2025-01-01  08:00         2.661971
3  Itapuama  2025-01-01  12:00         2.579012
4  Itapuama  2025-01-01  16:00         2.156502
5  Itapuama  2025-01-01  20:00         2.068936


In [79]:
df_result = prever_mare_auto('Sao Luis', '2025-01-01', df, model)
print(df_result)

      praia        data   hora  altura_prevista
0  Sao Luis  2025-01-01  00:00         1.610833
1  Sao Luis  2025-01-01  04:00         1.607059
2  Sao Luis  2025-01-01  08:00         2.661971
3  Sao Luis  2025-01-01  12:00         2.579012
4  Sao Luis  2025-01-01  16:00         2.156502
5  Sao Luis  2025-01-01  20:00         2.068936


In [80]:
import pickle

with open("modelo_xgb.pkl", "wb") as f:
    pickle.dump(model, f)
print("Modelo salvo como 'modelo_xgb.pkl'")

Modelo salvo como 'modelo_xgb.pkl'
